In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("s").getOrCreate()

In [3]:
data = spark.read.csv('data/Ecommerce_Customers.csv',inferSchema=True , 
                   header=True)

In [4]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [8]:
data.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [7]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
# this creates a column called features "ouput col" which contain the values of these columns as vector
assembler = VectorAssembler(inputCols = ['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol= 'features')

In [10]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [17]:
output.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))

In [18]:
final_data = output.select(['features','Yearly Amount Spent'])

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data , test_data = final_data.randomSplit([0.7,0.3])

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                361|
|   mean|  498.3258429217169|
| stddev|  80.29917477057559|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [24]:
from pyspark.ml.regression import LinearRegression

In [26]:
# features col defualt name is features
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [27]:
lr_model = lr.fit(train_data)

In [28]:
test_results = lr_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|0.043492151727321016|
| -12.899748809012806|
|  -3.404006558488163|
|    3.26448589198219|
|  18.782459679078386|
|  0.3094620752403898|
| -3.5059914633009157|
|  -8.457673641023234|
|   4.500369480422307|
| -1.8792566189689524|
|  -7.509187346833073|
|  -3.583055896367341|
| -2.3372176270856357|
|   8.678123695835495|
|  -6.284209630315388|
|  -9.048689101272146|
| -17.712338307163918|
|  -4.644700606551282|
|-0.07722522041132152|
|  -8.496140410839871|
+--------------------+
only showing top 20 rows



In [31]:
test_results.rootMeanSquaredError

9.877271264018406

In [32]:
test_results.r2

0.9833933226143012

In [33]:
unlabeled_data = test_data.select('features')

In [34]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.5743636841713...|
|[31.0662181616375...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3123495994443...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.4474464941278...|
|[31.5316044825729...|
|[31.5761319713222...|
|[31.7207699002873...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8512531286083...|
|[31.8745516945853...|
|[31.8854062999117...|
|[31.9563005605233...|
|[31.9673209478824...|
|[32.0047530203648...|
|[32.0085045178551...|
+--------------------+
only showing top 20 rows



In [35]:
prediction = lr_model.transform(unlabeled_data)

In [37]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|442.02092160633833|
|[31.0662181616375...|461.83304201668716|
|[31.2681042107507...| 426.8745397323121|
|[31.3091926408918...|429.45623194795144|
|[31.3123495994443...|444.80895834886223|
|[31.3895854806643...| 409.7601489847425|
|[31.4252268808548...| 534.2727101180628|
|[31.4474464941278...|427.06041573624725|
|[31.5316044825729...|432.01523624894025|
|[31.5761319713222...| 543.1058406082973|
|[31.7207699002873...|  546.284120824856|
|[31.8124825597242...|396.39340088016456|
|[31.8186165667690...| 448.7558909972213|
|[31.8512531286083...| 464.3141229709629|
|[31.8745516945853...| 398.5694538765829|
|[31.8854062999117...|399.15196207374765|
|[31.9563005605233...| 564.8382700543627|
|[31.9673209478824...|450.39454184620354|
|[32.0047530203648...|463.82320634104076|
|[32.0085045178551...|451.69336143959526|
+--------------------+------------